In [1]:
import numpy as np
from scipy import ndimage
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import gaussian_process
from sklearn.cross_validation import cross_val_score, KFold, train_test_split
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [2]:
num_images = 319
images = np.empty([1,20000])

In [3]:
for i in range(1,num_images+1):
    filename = "gray_images_all/image" + str(i) + ".jpg"
    img = ndimage.imread(filename)
    if (i == 1):
        images = img.flatten()
    else:
        images = np.vstack((images, img.flatten()))
        
images = images.astype(float)
print images.shape
print images.min()
print images.max()

(319, 20000)
0.0
255.0


In [4]:
scaler = preprocessing.StandardScaler().fit(images)
images_scaled = scaler.transform(images)
print images_scaled.shape
print images_scaled.min()
print images_scaled.max()

(319, 20000)
-17.8325545001
17.8325545001


In [5]:
thresh_components = 10
n_components = thresh_components
while(True):
    pca0 = decomposition.PCA(n_components=n_components)
    pca0.fit(images_scaled)
    if pca0.explained_variance_ratio_.sum() > 0.9:
        break
    n_components += thresh_components

In [6]:
pca = decomposition.PCA(n_components=n_components)
pca.fit(images_scaled)
data = pca.transform(images_scaled)
print data.shape
print data.min()
print data.max()

(319, 100)
-102.788361097
202.391869066


In [7]:
print n_components
print pca.explained_variance_ratio_.sum()

100
0.902236779604


In [8]:
for i in range(len(data)-2):
    newrow = np.hstack((data[i], data[i+1]))
    if (i == 0):
        X = newrow
    else:
        X = np.vstack((X, newrow))

In [9]:
print X.shape
ylist = []
for i in range(2,len(data)):
    ylist.append(data[i])
y = np.asarray(ylist)
print y.shape

(317, 200)
(317, 100)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=0)
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(190, 200)
(190, 100)
(127, 200)
(127, 100)


In [11]:
gp = gaussian_process.GaussianProcess(theta0=1e-2, thetaL=1e-4, thetaU=1e-1)
gp.fit(X_train, y_train)

GaussianProcess(beta0=None,
        corr=<function squared_exponential at 0x7f4ca1f76a28>,
        normalize=True, nugget=array(2.220446049250313e-15),
        optimizer='fmin_cobyla', random_start=1,
        random_state=<mtrand.RandomState object at 0x7f4cb023e890>,
        regr=<function constant at 0x7f4ca1f76668>, storage_mode='full',
        theta0=array([[ 0.01]]), thetaL=array([[ 0.0001]]),
        thetaU=array([[ 0.1]]), verbose=False)

In [12]:
print X_test[0].shape
print X_test[0][:pca.n_components_].shape

(200,)
(100,)


In [13]:
mse = []
y_pred_list=[]

for i in range(len(X_test)-2):
    if(i == 0):
        X_test_sample = X_test[0]
    else:
        X_test_sample = np.hstack((X_test[i][:pca.n_components_], y_test_pred[0]))
    y_test_pred, mse_pred = gp.predict(X_test_sample, eval_MSE=True)
    y_pred_list.append(y_test_pred[0])
    mse.append(mse_pred)

y_pred = np.asarray(y_pred_list)
y_pred_size = y_pred.shape[0]
y_test_size = y_test.shape[0]
print y_pred.shape
print len(mse)
print y_test.shape

(125, 100)
125
(127, 100)


In [14]:
inv_y_pred = pca.inverse_transform(y_pred)
print inv_y_pred.shape
inv_y_test = pca.inverse_transform(y_test)
print inv_y_test.shape

(125, 20000)
(127, 20000)


In [15]:
original_y_pred = scaler.inverse_transform(inv_y_pred)
original_y_test = scaler.inverse_transform(inv_y_test[2:y_test_size])
print original_y_pred.shape
print original_y_test.shape

(125, 20000)
(125, 20000)


In [16]:
print 'inverted :', inv_y_pred[0].min(), inv_y_pred[0].max()
print 'original :', original_y_pred[0].min(), original_y_pred[0].max()

inverted : -0.956429197413 1.11501173434
original : -0.0951052265815 253.687595839


In [17]:
pred_image = original_y_pred.reshape((y_pred_size, 100, 200))
original_image = original_y_test.reshape((y_pred_size, 100, 200))

In [18]:
print pred_image[0].shape
print original_image[0].shape

(100, 200)
(100, 200)


In [19]:
error_image = abs(original_image - pred_image)
print error_image.shape

(125, 100, 200)


In [20]:
for i in range(y_pred_size):
    fig = plt.figure()
    a = fig.add_subplot(1,3,1)
    orig = plt.imshow(original_image[i], cmap = cm.Greys_r)
    a.set_title('Orig')
    a = fig.add_subplot(1,3,2)
    pred = plt.imshow(pred_image[i], cmap = cm.Greys_r)
    a.set_title('Pred')
    a = fig.add_subplot(1,3,3)
    pred = plt.imshow(error_image[i], cmap = cm.Greys_r)
    a.set_title('Error')
    #plt.show()
    fig.savefig('output_images_all_x1y1/output_'+str(i+1)+'.jpg')
    fig.clear()

/home/sourav/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:424: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
